In [154]:
# dependencies
import json
import pandas as pd
import re

# load types file
with open('../API_Data/pkmn.json', 'r') as in_file:
    pkmn = json.load(in_file)

In [159]:
pkmn_names = pd.Series([pkmn[num]['name'] for num in range(len(pkmn))])

Here I'm going to want to change out form names to something that looks a little better (ex: raichu-alola -> Alolan Raichu)

In [160]:
def clean_names(name):
    
    # drop gigantimaxes, pikachu and eevee costumes, totems
    to_drop = ['gmax', 'pikachu-', 'eevee-', 'totem', 'floette-', 'greninja-', 'magearna-', 'cramorant-', 'zarude-']
    for pattern in to_drop:
        if re.match(f'.*{pattern}', name, flags = re.IGNORECASE):
            name = 'DROP'
            return name.title()

    # capture all region-specific forms
    regions = ['alol', 'galar', 'hisui']
    for region in regions:
        if re.match(f'.*{region}', name, flags=re.IGNORECASE):
            
            new = re.search(f'(.*)-{region}-?(.*)?', name)
            if new:
                if region in ['alol', 'hisui']:
                    name = f'{region}an {new.group(1).replace("-", " ")}'
                elif region in ['galar']:
                    if new.group(2):
                        name = f'{region}ian {new.group(1).replace("-", " ")} ({new.group(2).replace("-", " ")})'
                    else:
                        name = f'{region}ian {new.group(1).replace("-", " ")}'
            return name.title()
    
    # minior (just keep one color since they're all the same)
    if re.match(f'minior-.*', name, flags = re.IGNORECASE):
        new = re.search(f'minior-(.*)', name)
        if new.group(1) == 'red':
            name = f'minior (core form)'
        elif new.group(1) == 'red-meteor':
            name = f'minior (meteor form)'
        else:
            name = 'DROP'
        return name.title()    
    
    # pokemon with different formes
    forme_search = ['deoxys', 'dialga', 'palkia', 'giratina', 'shaymin', 'tornadus', 'thundurus', 'landorus', 'meloetta', 'aegislash', 'zygarde', 'enamorus']
    for pattern in forme_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} ({new.group(1).replace("-", " ")} forme)'
            return name.title()
    
    # pokemon with different forms
    form_search = ['castform', 'basculin', 'keldeo', 'lycanroc', 'wishiwashi', 'toxtricity']
    for pattern in form_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} ({new.group(1).replace("-", " ")} form)'
            return name.title()

    # pokemon with different styles
    style_search = ['oricorio', 'urshifu']
    for pattern in style_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} ({new.group(1).replace("-", " ")} style)'
            return name.title()

    # replace dash with space
    poke_search = ['tapu']
    for pattern in poke_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} {new.group(1).replace("-", " ")}'
            return name.title()
    
    # replace dash with space with () around modifier
    modifier_search = ['mimikyu', 'eiscue', 'kyurem', 'kyogre', 'groudon', 'hoopa', 'rockruff', 'necrozma', 'eternatus', 'darmanitan', 'zacian', 'zamazenta', 'calyrex', 'morpeko', 'pumpkaboo', 'gourgeist']
    for pattern in modifier_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} ({new.group(1).replace("-", " ")})'
            return name.title()
    
    # pokemon with different genders
    gender_search = ['m', 'male', 'f', 'female']
    for pattern in gender_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern})'
            return name.title()
    
    # pokemon with different cloaks (burmy and wormadam)
    cloak_search = ['plant', 'sandy', 'trash']
    for pattern in cloak_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern} cloak)'
            return name.title()
    
    # rotom
    if re.match(f'rotom-.*', name, flags = re.IGNORECASE):
        new = re.search(f'rotom-(.*)', name)
        name = f'{new.group(1)} rotom'
        return name.title()

    # mr [mr]imes
    mime_search = ['mr-mime', 'mime-jr', 'mr-rime']
    for pattern in mime_search:
        if re.match(f'{pattern}', name, flags = re.IGNORECASE):
            new = re.search(f'({pattern})', name)
            name = new.group(1).replace("-", " ")
            return name.title()
    
    # type: null
    if re.match('type-null', name, flags = re.IGNORECASE):
        new = re.search(f'(.*)-(.*)', name)
        name = f'{new.group(1)}: {new.group(2)}'
        return name.title()

    # capture all mega evolutions
    if re.match('.*mega', name, flags=re.IGNORECASE):
        
        if re.match('.*-mega$', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega', name)
            if new:
                name = 'mega ' + new.group(1)
        elif re.match('.*-mega-[xy]', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega-([xy])', name)
            if new:
                name = 'mega ' + new.group(1) + ' ' + new.group(2)
            return name.title()
        
    return name.title()

In [161]:
pkmn_names = pkmn_names.apply(clean_names)

In [162]:
for name in pkmn_names[pkmn_names.str.contains(r'-')]:
    print(name)

Ho-Oh
Porygon-Z
Jangmo-O
Hakamo-O
Kommo-O
